# BLIP

In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests

# Load model and processor
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

# Load your image


c:\Users\fareh\coding\video-rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\fareh\coding\video-rag\.venv\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\fareh\.cache\huggingface\hub\models--Salesforce--blip-image-captioning-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In o

BLIP Caption: there is a small cat walking in the snow near a fence


In [4]:
url = r"C:\Users\fareh\coding\video-rag\sample-2.png"
image = Image.open(url).convert('RGB')

# Generate Caption
inputs = processor(image, return_tensors="pt")
out = model.generate(**inputs)
print("BLIP Caption:", processor.decode(out[0], skip_special_tokens=True))

BLIP Caption: a screenshot of a whiteboard with a drawing of a graph


# QWEN2.5VL

In [ ]:
pip install qwen-vl-utils

In [11]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": url},
            {"type": "text", "text": "Describe this image in great detail."},
        ],
    }
]

# Preparation
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, _ = process_vision_info(messages)
inputs = processor(text=[text], images=image_inputs, return_tensors="pt").to("cuda")

# Inference
generated_ids = model.generate(**inputs, max_new_tokens=128)
output_text = processor.batch_decode(generated_ids, skip_special_tokens=True)
print("Qwen2.5-VL:", output_text[0])

ModuleNotFoundError: No module named 'torchvision'

# FLORENCE-2

In [ ]:
from PIL import Image
image = Image.open("../image.png").convert("RGB")

In [ ]:
import requests

import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM 


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large", torch_dtype=torch_dtype, trust_remote_code=True).to(device)
processor = AutoProcessor.from_pretrained("microsoft/Florence-2-large", trust_remote_code=True)

prompt = "<OD>"

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/transformers/tasks/car.jpg?download=true"
image = Image.open(requests.get(url, stream=True).raw)

inputs = processor(text=prompt, images=image, return_tensors="pt").to(device, torch_dtype)

generated_ids = model.generate(
    input_ids=inputs["input_ids"],
    pixel_values=inputs["pixel_values"],
    max_new_tokens=4096,
    num_beams=3,
    do_sample=False
)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]

parsed_answer = processor.post_process_generation(generated_text, task="<OD>", image_size=(image.width, image.height))

print(parsed_answer)


AttributeError: module 'torch' has no attribute 'cuda'

# MOLMO 7B

In [ ]:
pip install einops

In [ ]:

from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig

processor = AutoProcessor.from_pretrained('allenai/Molmo-7B-D-0924', trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained('allenai/Molmo-7B-D-0924', trust_remote_code=True, device_map="auto")

inputs = processor.process(images=[image], text="Describe this image.")
inputs = {k: v.to(model.device).unsqueeze(0) for k, v in inputs.items()}

output = model.generate_from_batch(
    inputs, 
    GenerationConfig(max_new_tokens=200, stop_strings="<|endoftext|>"),
    tokenizer=processor.tokenizer
)

generated_text = processor.tokenizer.decode(output[0, inputs['input_ids'].size(1):], skip_special_tokens=True)
print("Molmo:", generated_text)

# QWEN2.5VL 3B

In [3]:
uv pip install accelerate

Note: you may need to restart the kernel to use updated packages.


c:\Users\Owais\Downloads\video-rag\.venv\Scripts\python.exe: No module named uv


In [ ]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
import torch

# 4-bit quantization is the "secret sauce" for your 4GB card
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct",
    quantization_config=bnb_config,
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

c:\Users\Owais\Downloads\video-rag\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Owais\Downloads\video-rag\.venv\lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Owais\.cache\huggingface\hub\models--Qwen--Qwen2.5-VL-3B-Instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order 

In [ ]:
from PIL import Image


messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

inputs = processor(
    messages,
    return_tensors="pt"
).to(device)

with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=128
    )

output_text = processor.batch_decode(
    output_ids, skip_special_tokens=True
)[0]

print(output_text)


## Gemma 3

In [1]:
import re

def clean_response(text):
    return re.sub(r"^\s+|\s+$|\*\*|[\r\n\t]+|\s+", " ", text).lower()


zero shot

In [3]:
from ollama import chat
from PIL import Image
# from pathlib import Path

# Pass in the path to the image
#image=Image.open(r'C:\Users\fareh\coding\video-rag\image.png').convert('RGB')
path = r'C:\Users\fareh\coding\video-rag\image.png'
path2=r'C:\Users\fareh\coding\video-rag\sample-2.png'


response = chat(
  model='gemma3',
  messages=[
      {
  "role": "system",
  "content": "You are a helpful assistant that can analyze images and provide captions"
},

    {
      'role': 'user',
      'content': 'What is in this image? .',
      'images': [path2],
    }
  ],
)

print(clean_response(response.message.content))

ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download

chain of thought

In [4]:
from ollama import chat
# from pathlib import Path

# Pass in the path to the image
path = r'C:\Users\fareh\coding\video-rag\image.png'


response = chat(
  model='gemma3',
  messages=[
      {
          'role':'system',
          'content':'''
You are a visual reasoning assistant.
When given an image, first describe the important objects,
then reason step by step, and finally give a concise answer.
Do not hallucinate details that are not visible.
Final Caption: A blurry photo of a city street with buildings. '''
      },
    {
      'role': 'user',
      'content': 'What is in this image? .',
      'images': [path],
    }
  ],
)

print(clean_response(response.message.content)  )

here's a breakdown of the image:  description:  the image shows a man with fair skin, dark hair, and a blue shirt, seemingly reacting in a moment of frustration. he’s pointing upwards with his right hand. in the center of the frame, against a black background, is the integral symbol ∫ followed by the expression x⁻¹.  reasoning:  the image is a still frame from a video. the man is likely demonstrating or reacting to a problem in mathematics, specifically an integral calculation. the integral symbol and the expression x⁻¹ are clearly visible, indicating the focus of the video content.  final caption:  a still image of a man with a facial expression of frustration in front of the integral symbol ∫x⁻¹.


object detection 

In [5]:
from ollama import chat
from pydantic import BaseModel
from typing import Literal, Optional

path ='C:\\Users\\fareh\\coding\\video-rag\\sample-2.png'
class Object(BaseModel):
  name: str
  confidence: float
  attributes: str

class ImageDescription(BaseModel):
  summary: str
  objects: list[Object]
  scene: str
  colors: list[str]
  time_of_day: Literal['Morning', 'Afternoon', 'Evening', 'Night']
  setting: Literal['Indoor', 'Outdoor', 'Unknown']
  text_content: Optional[str] = None

response = chat(
  model='gemma3',
  messages=[{
    'role': 'user',
    'content': 'Describe this photo and list the objects you detect.',
    'images': [path]
  }],
  format=ImageDescription.model_json_schema(),
  options={'temperature': 0},
)

image_description = ImageDescription.model_validate_json(response.message.content)
print(clean_response(str(image_description)))

RemoteProtocolError: Server disconnected without sending a response.

In [ ]:
from ollama import chat
from PIL import Image
# from pathlib import Path

# Pass in the path to the image
#image=Image.open(r'C:\Users\fareh\coding\video-rag\image.png').convert('RGB')
path = r'C:\Users\fareh\coding\video-rag\sample-2.png'


response = chat(
  model='gemma3',
  messages=[
      {
  "role": "system",
  "content": '''You are analyzing a frame from a video.

Describe:
- what is happening in this moment
- who or what is present
- the location or setting
- objects involved in the activity

# Write one descriptive sentence suitable for search and retrieval.
'''
},

    {
      'role': 'user',
      'content': 'What is in this image? .',
      'images': [path],
    }
  ],
)

print(clean_response(response.message.content))
#print(response.message.content)

instead of genrating than cleaning => cleaning parallel with streaming 

In [ ]:
from ollama import chat

stream = chat(
  model='gemma3',
  messages=[{'role': 'user', 'content': 'What is 17 × 23?'}],
  stream=True,
)

in_thinking = False
content = ''
thinking = ''
for chunk in stream:
  if chunk.message.thinking:
    if not in_thinking:
      in_thinking = True
      print('Thinking:\n', end='', flush=True)
    print(chunk.message.thinking, end='', flush=True)
    # accumulate the partial thinking 
    thinking += chunk.message.thinking
  elif chunk.message.content:
    if in_thinking:
      in_thinking = False
      print('\n\nAnswer:\n', end='', flush=True)
    print(chunk.message.content, end='', flush=True)
    # accumulate the partial content
    content += chunk.message.content

  # append the accumulated fields to the messages for the next request
  new_messages = [{ 'role': 'assistant', 'thinking': thinking, 'content': content }]

17 × 23 = (10 + 7) × (20 + 3)
= 10 × 20 + 10 × 3 + 7 × 20 + 7 × 3
= 200 + 30 + 140 + 21
= 200 + 30 + 140 + 21
= 230 + 161
= 391

Alternatively, we can use the standard multiplication method:
   17
 x 23
 ------
   51
+340
 ------
  391

Therefore, 17 × 23 = 391.

Final Answer: The final answer is $\boxed{391}$

In [ ]:

!hf auth login

User is already logged in.


In [ ]:
import torch
from transformers import pipeline

pipeline = pipeline(
    task="image-text-to-text",
    model="google/gemma-3-4b-pt",
    device=0,
    dtype=torch.bfloat16
)
pipeline(
    "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/pipeline-cat-chonk.jpeg",
    text="<start_of_image> What is shown in this image?"
)

c:\Users\fareh\coding\video-rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-3-4b-pt.
401 Client Error. (Request ID: Root=1-69884615-3fa9c5d331cd779a0324bc4b;d0cb519e-3eaf-46ed-9546-292f698e0339)

Cannot access gated repo for url https://huggingface.co/google/gemma-3-4b-pt/resolve/main/config.json.
Access to model google/gemma-3-4b-pt is restricted. You must have access to it and be authenticated to access it. Please log in.